## Sentiment Analysis 

In [1]:
%matplotlib inline
import matplotlib
import utils
import importlib
importlib.reload(utils)
from utils import *
import re, string, collections, pickle, os # bcolz
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB

import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

## Read data

In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\repository\product3team9\data\cleaned\funda_with_buurt.csv')
print(df.shape)
df.head()

(188262, 37)


,globalId,publicatieDatum,postcode,koopPrijs,volledigeOmschrijving,categorieObject,indTuin,perceelOppervlakte,kantoor_naam_MD5hash,aantalKamers,...,Violence_and_sexual_abuse,NBH_code_y,%_low_income_households,%_below_social_minimum,NBH_code_x.1,No_residents,Density,NBH_code_y.1,0-4age_Binnen5Km_55,4+age_Binnen10Km
0,4388200,2018-09-24,7481LK,209000.0,Verrassend ruime tussenwoning nabij het centru...,Woonhuis,1,148.0,6A91BF7DB06A8DF2C9A89064F28571E7,5,...,3.0,1580001,6.0,6.0,1580001,1955.0,4.0,1580001,8.0,5.0
1,4399344,2018-08-02,1068MS,267500.0,- ENGLISH TRANSLATION - \n\nOn the 21st of Sep...,Appartement,0,70.0,E983FEDC63D87BF61AE952D181C8FD17,3,...,1.0,3638103,1.0,1.0,3638103,1245.0,4.0,3638103,1.0,11.0
2,4401831,2018-08-06,5971CR,162500.0,"In een rustige wijk, op korte afstand van het ...",Woonhuis,1,104.0,DA6EDCA2E6F7AADE8D9817099455ABC4,4,...,0.0,15070802,1.0,1.0,15070802,800.0,3.0,15070802,4.0,14.0
3,4402098,2018-08-06,9571BM,217500.0,In landelijke woonomgeving en aan de doorgaand...,Woonhuis,1,1028.0,FB71E2057357FAC18F2CDB18C3F15FC2,5,...,6.0,16811500,6.0,5.0,16811500,2360.0,6.0,16811500,3.0,0.0
4,4406997,2018-08-12,1031KA,655000.0,Dit betreft bouwnummer 24 van het project Aan ...,Appartement,0,105.0,F8471E80DFB18392B3D1AA2BFD2C1CE4,3,...,1.0,3637107,4.0,3.0,3637107,1360.0,4.0,3637107,9.0,13.0


In [8]:
sample=df.iloc[0:100]

## Format data

In [9]:
import re
# We just keep 2 columns: text and score
sample = sample[['volledigeOmschrijving', 'globalId']]
sample['volledigeOmschrijving'] = sample['volledigeOmschrijving'].str.replace(' \n\n',' ')
sample['volledigeOmschrijving'] = sample['volledigeOmschrijving'].str.replace('\n','. ')
sample['volledigeOmschrijving'] = sample['volledigeOmschrijving'].str.replace('\d+','')

for row in range(len(sample['volledigeOmschrijving'])):
    sample['volledigeOmschrijving'].iloc[row] = re.sub('[^.,a-zA-Z0-9 \n\.]', '',sample['volledigeOmschrijving'].iloc[row])

## Applying sentence tokenizer

In [10]:
from nltk.tokenize import sent_tokenize
sample['volledigeOmschrijving'].dropna(inplace=True)
sample['volledigeOmschrijving'] = sample['volledigeOmschrijving'].apply(sent_tokenize)

In [22]:
magic=sample.copy()

## Language detection for every sentence

In [23]:
import numpy
import re
from langdetect import detect, detect_langs
for row in magic['volledigeOmschrijving']:
    row= numpy.reshape(row, ((len(row)),1)).tolist()
    for col in row:
        if re.search('[a-zA-Z]', col[0]):
            lang_prop=(detect(col[0]))
            col.append(lang_prop)

## Appending only Dutch description

In [26]:
clean_desc = []
for desc in range(len(magic)):
    temp_list = []
    for lst in range(len(magic['volledigeOmschrijving'].iloc[desc])):
        if magic['volledigeOmschrijving'][desc][lst][0] != '' and magic['volledigeOmschrijving'][desc][lst][0] != '.':
            if magic['volledigeOmschrijving'].iloc[desc][lst][1] != 'en':
                temp_list.append(other['volledigeOmschrijving'].iloc[desc][lst])
    clean_desc.append(temp_list)                    
clean_desc

[['Verrassend ruime tussenwoning nabij het centrum.',
  'Het lijkt een gewone tussenwoning.',
  'Schijn bedriegt, want eenmaal binnen ontdek je verrassend veel ruimte.',
  'Dat dachten wij ook toen we dit huis twintig jaar geleden voor het eerst zagen, vertellen de huidige bewoners.',
  'Privacy, zowel vr als achter We zitten hier niet in de etalage.',
  'De ligging nabij hartje Haaksbergen, met alle winkels dichtbij.',
  'En tegelijkertijd de rust hier aan het eind van een doodlopend straatje.',
  'Dat gaf destijds de doorslag om hier te gaan wonen.. .',
  'Buurt .',
  'Aan de straatkant heeft het huis een aangebouwde stenen berging, die ook bereikbaar is vanuit de hal.',
  'De dakkapel en verder alle ramen, kozijnen en buitendeuren behalve die van de berging zijn van kunststof.',
  'Beweegbare ramen zijn draaikiep.',
  'Er is een eigen oprit en bovendien voldoende parkeerplekken in de directe omgeving.',
  'Aan de overzijde van de straat ligt een groot en groen openbaar terrein dat e

In [27]:
clean_desc_df = pd.DataFrame(clean_desc)
clean_desc_df

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,Verrassend ruime tussenwoning nabij het centrum.,Het lijkt een gewone tussenwoning.,"Schijn bedriegt, want eenmaal binnen ontdek je...",Dat dachten wij ook toen we dit huis twintig j...,"Privacy, zowel vr als achter We zitten hier ni...","De ligging nabij hartje Haaksbergen, met alle ...",En tegelijkertijd de rust hier aan het eind va...,Dat gaf destijds de doorslag om hier te gaan w...,Buurt .,Aan de straatkant heeft het huis een aangebouw...,...,None,None,None,None,None,None,None,None,None,None
1,ENGLISH TRANSLATION On the st of September y...,Bidding closes on st September at PM.,Viewings can be arranged via the realtor as me...,Please notice that you will need to register y...,Biedtermijn sluit op september om uur Bezic...,Houd er rekening mee dat je je eerst moet regi...,GROUNDLEASE.,"There is a ground lease, which is around .",per year.,DUTCH INFORMATION Dit mooie kamerappartement ...,...,None,None,None,None,None,None,None,None,None,None
2,"In een rustige wijk, op korte afstand van het ...",Het terras is gelegen op het noordoosten.. .,Kijk voor de video op de site van de makelaar.,Indeling.,Entree.,Vanuit de entree heeft u toegang tot de woonka...,Woonkamer.,De woonkamer met nis heeft dezelfde tegelvloer...,De wanden zijn afgewerkt middels spachtelputz ...,Door de grote raampartij aan de voorzijde is h...,...,None,None,None,None,None,None,None,None,None,None
3,In landelijke woonomgeving en aan de doorgaand...,De woning met aangebouwde garage is gebouwd in...,De woning is traditioneel gebouwd en v.v.,"vloer, muur, en glasisolatie.",I.p.v dakisolatie is plafondisolatie aanwezig ...,Deze woning is zeer geschikt voor zowel senior...,Indeling.,begane grond.,"halentree, meterkast, toilet met fontein, slaa...",", m x .",...,None,None,None,None,None,None,None,None,None,None
4,Dit betreft bouwnummer van het project Aan he...,Het appartementencomplex van KCAP in project A...,Het gebouw kenmerkt zich door een getrapte gev...,Deze bieden veel privacy en de gemeenschappeli...,Het appartementencomplex is met oog voor detai...,Perfect op elkaar afgestemde materialen Jura ...,"De hoogwaardige, ranke kozijnen zijn van geano...",Elk appartement heeft een fijn balkon omringd ...,Appartementen .,De appartementen hebben een slimme plattegrond...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"In een rustige woonomgeving, met aan de voorzi...","Woning is ruim van opzet, goed gesoleerd en be...",Indeling.,Begane grond.,"Halentree met tegelvloer, meterkast, trapkast ...",L vormige woonkamer met tegelvloer en open ke...,Keuken in een eiken uitvoering met diverse app...,"gaskookplaat, oven, afzuiging en koelkast.",Vanuit de keuken alsmede de serre toegang naar...,Vanuit de bijkeuken toegang naar de oprit.,...,None,None,None,None,None,None,None,None,None,None
96,Royaal uitgebouwde hoekwoning met twee garages...,"De woning beschikt over een grote, uitgebouwde...",Wat opvalt bij deze woning is de extra garage ...,Dit betreft een dubbele garage en is voorzien ...,Ook is een garage naast de woning gerealiseerd.,U krijgt dus de beschikking over enorm veel be...,Ideaal voor de ZZP er of iemand die gewoon vee...,Indeling.,Oprit naar garagecarport met elektrisch bedien...,Begane grond hal met toilet met fontein en me...,...,None,None,None,None,None,None,None,None,None,None
97,"Multifunctioneel WOONWINKEL PAND te koop, gele...",Dit ruimbemeten object is gelegen op m eigen ...,Het bestemmingsplan centrumdoeleinden laat hie...,Wij denken graag met u mee over de talrijke mo...,Indeling.,Souterrain.,Kelder bergruimte en opslag.,Parterre.,Winkel .,Via entree direct in winkelruimte gedeelte ca ...,...,None,None,None,None,None,None,None,None,None,None
98,Wonen n werken op n adres .,Van harte welkom bij deze vrijstaande bedrijfs...,HALENTREE.,Wat een welkom Een ruime hal met trapopgang na...,In de hal be

In [28]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  
        
        
# Driver code     
for row in range(len(clean_desc)):
    clean_desc[row] = listToString(clean_desc[row])

In [29]:
clean_desc_df = pd.DataFrame(clean_desc)
clean_desc_df['description'] = clean_desc_df[0]
clean_desc_df = clean_desc_df.drop([0],axis = 1)
clean_desc_df['description'] = clean_desc_df['description'].astype(str)

### Tokenize & Lemmatize (extract clean words)

In [32]:
# Backup
df_raw = clean_desc_df.copy()
# df = df_raw

In [33]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('dutch'))
lemmatizer = WordNetLemmatizer()

for i in range(df_raw.shape[0]):
    clean_tokens = ''
    for w in word_tokenize(df_raw.loc[i, ('description')]):
        if w not in stop_words:
            clean_tokens = clean_tokens + ' ' + lemmatizer.lemmatize(w)
    df_raw.loc[i, ('description')] = clean_tokens.translate(str.maketrans('', '', string.punctuation))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zuzan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Sentiment & Subjectivity calculation

In [35]:
from pattern.nl import sentiment
sentiments=[]
import operator
for desc in range(df_raw.shape[0]):
    #Sentiment calculation based on a summary field. Sentiment function will output in 2 values - sentiment & objectivity. For the analysis purposes only sentiment is needed and thus [0]
    sentiments.append(sentiment(df_raw.description.iloc[desc]))
    
#column Sentiment should include an array of sentiments
df_raw['sentiment']= list(map(operator.itemgetter(0),sentiments))
df_raw['subjectivity']= list(map(operator.itemgetter(1),sentiments))

In [36]:
df_raw

,description,sentiment,subjectivity
0,Verrassend ruime tussenwoning nabij centrumHe...,0.242772,0.593986
1,ENGLISH TRANSLATION On the st September you w...,0.302244,0.639808
2,In rustige wijk korte afstand centrum Grubbe...,0.223125,0.486250
3,In landelijke woonomgeving doorgaande weg sta...,0.211042,0.542813
4,Dit betreft bouwnummer project Aan IJ The Gri...,0.291860,0.574767
...,...,...,...
95,In rustige woonomgeving voorzijde weids uitz...,0.148000,0.500400
96,Royaal uitgebouwde hoekwoning twee garage De...,0.297222,0.590972
97,Multifunctioneel WOONWINKEL PAND koop gelege...,0.302000,0.548889
98,Wonen n werken n adres Van harte welkom vrijs...,0.321944,0.612063


## Topic Modelling

In [37]:
import numpy as np, pandas as pd
%matplotlib inline
import matplotlib
import re, string, collections, pickle, os

import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
# from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## Lemmatize and Tokenize

In [38]:
# Functions to lemmatize using POS tags
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('dutch'))

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB,
                "R": wn.ADV}
    return tag_dict.get(tag, wn.NOUN)

# Includes removal of stop words and punctuation
def get_clean_token(txt):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_tokenize(txt.translate(str.maketrans('.', ' ', string.punctuation))) if w not in stop_words]

In [39]:
# Vectorizer with options to filter the words
vectorizer = CountVectorizer(tokenizer=get_clean_token, # This function includes the lemmatization using pos tags
                             min_df=5, max_df=0.9, # min-max word frequency (to select not too rare & not too common words)
                             lowercase=True, 
                             token_pattern='[a-zA-Z][a-zA-Z]{2,}') # Ignore words containing numbers, special characters, or containing less than 2 characters.

data_vectorized = vectorizer.fit_transform(df_raw['description'])

In [40]:
data_vectorized

<100x874 sparse matrix of type '<class 'numpy.int64'>'
	with 11024 stored elements in Compressed Sparse Row format>

In [41]:
# Build a Latent Dirichlet Allocation model
lda_model = LatentDirichletAllocation(n_components=5, # Detect 5 topic
                                      max_iter=10, 
                                      learning_method='online') # Option for large dataset, or else use learning_method='batch'
lad_Z = lda_model.fit_transform(data_vectorized)

In [42]:
lad_Z

array([[1.24324899e-03, 9.95001729e-01, 1.23716045e-03, 1.25299453e-03,
        1.26486750e-03],
       [1.85180036e-03, 8.25448961e-01, 1.80685289e-03, 1.69082205e-01,
        1.81018162e-03],
       [1.55677609e-03, 9.93756692e-01, 1.56166502e-03, 1.57250178e-03,
        1.55236481e-03],
       [1.97336846e-03, 9.92089838e-01, 1.98168811e-03, 1.98127506e-03,
        1.97383006e-03],
       [9.95681934e-01, 1.08350130e-03, 1.07582973e-03, 1.08085265e-03,
        1.07788212e-03],
       [9.84870411e-04, 9.96055732e-01, 9.81104638e-04, 9.85258446e-04,
        9.93034139e-04],
       [1.88894441e-03, 9.92470926e-01, 1.87536906e-03, 1.89190029e-03,
        1.87285994e-03],
       [1.82959117e-03, 6.35253359e-01, 1.82758390e-03, 3.59213767e-01,
        1.87569879e-03],
       [1.37710413e-03, 9.94489602e-01, 1.38315843e-03, 1.37941285e-03,
        1.37072217e-03],
       [1.67900759e-03, 9.93299908e-01, 1.66807019e-03, 1.68367201e-03,
        1.66934189e-03],
       [1.58310738e-03, 9.9362

## Topic Modelling

In [44]:
# pip install pyLDAvis
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  69.431389
3       51.163364 -159.968018       2        1  18.050453
4      -73.144554   21.523815       3        1   7.474113
0      -66.410133 -110.606438       4        1   3.781893
2       54.417324   12.443444       5        1   1.262152, topic_info=         Term        Freq       Total Category  logprob  loglift
402    keuken  172.000000  172.000000  Default  30.0000  30.0000
51   badkamer  139.000000  139.000000  Default  29.0000  29.0000
828    woning  272.000000  272.000000  Default  28.0000  28.0000
623    ruimte   82.000000   82.000000  Default  27.0000  27.0000
471         m  279.000000  279.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
680   toegang    1.045925  110.775683   Topic5  -5.2433  -0.2903
354    houten    0.867356   46.557477   Topic5  -5.4305   0.3894
503     mooie    0.830939   48.951797   Topic5  -5.4734   0.2963
29       alle    0.853275   87.227132   Topic5  -5.4469  -0.2548
264   gelegen    0.824386  121.398151   Topic5  -5.4814  -0.6198

[318 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
0         1  0.726084                 a
0         2  0.258605                 a
0         3  0.009946                 a
2         1  0.111050        aangegeven
2         2  0.888398        aangegeven
...     ...       ...               ...
863       3  0.604725  zolderverdieping
864       1  0.325698               zon
864       2  0.651397               zon
864       3  0.032570               zon
873       3  0.845559        zwartkleur

[652 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 1, 3])